In [1]:
import pickle
from encode_utils.rerank_data import rerank_dist, rerank_single
from encode_utils.efficient_rerank import get_effrerank_model, run_comstyle
from encode_utils.sco_funct import weightaddprob, default_scofunct
from encode_utils.mt_scores import get_scores_auto
from transformers import AutoTokenizer
import pandas as pd
import numpy as np
import torch
import random
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


2022-12-22 08:01:17.905942: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-22 08:01:17.905962: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# setup / load model
encodemod = get_effrerank_model("noun")
xlm_tok = AutoTokenizer.from_pretrained("xlm-roberta-base")

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Encoder model frozen.
Loading weights from /mnt/data1/prasann/latticegen/lattice-generation/COMET/lightning_logs/version_44/checkpoints/epoch=9-step=40000.ckpt.


freeze embeds


In [4]:
# Issue #1 - changing pos-ids with identical strings changes token scores for all things
# score check (at token level) the best option, and the new option (should be same until they get to non-identical tokens)

In [8]:
# get token level scores from model
def get_hyp_sco_verb(inphyp, posids=None):
    
    tokens = xlm_tok(inphyp, return_tensors='pt').to(device)
    tokens = tokens.input_ids

    if posids is None: 
        # have to add 2 because of how xlm roberta works
        positionids = torch.arange(len(tokens[0])).unsqueeze(0).to(device)+2
        print(positionids)

    elif -1 in posids:
        positionids = None
    else:
        # get token at the end
        positionids = posids
        #torch.tensor(posids+[posids[-1]+1]).unsqueeze(0).to(device)
        print(positionids)

    # get a causal mask to use
    tmpmask = torch.tril(torch.ones(len(tokens[0]), len(tokens[0]))).unsqueeze(0).to(device)
    
    # this is a dummy string (using CQE model architecture which requires an "input" pass)
    toked_inp = xlm_tok(["noun"], return_tensors="pt").to(device)
    
    # run inputs through the model
    predout = encodemod(toked_inp.input_ids, toked_inp.attention_mask, tokens, positionids, \
        tmpmask)
    
    tmppred = predout['score']
    return tmppred, tokens, positionids, tmpmask


In [9]:
# get scores for identical strings, with different posids
sco_default = get_hyp_sco_verb("I am a bat")
modpos = sco_default[2]
# change last position id
modpos[0][5] = 6
sco_mod = get_hyp_sco_verb("I am a bat", modpos)

tensor([[2, 3, 4, 5, 6, 7]], device='cuda:0')
tensor([[2, 3, 4, 5, 6, 6]], device='cuda:0')


In [15]:
# these two should be the same except for the last 2 tokens, but they aren't
print(sco_default[0])
print(sco_mod[0])

tensor([[[0.0000],
         [0.0042],
         [0.0042],
         [0.0043],
         [0.0508],
         [0.0214]]], device='cuda:0', grad_fn=<DivBackward0>)
tensor([[[0.0000],
         [0.0049],
         [0.0049],
         [0.0050],
         [0.0593],
         [0.0138]]], device='cuda:0', grad_fn=<DivBackward0>)
